# Tratando dados do levantamento físico e banco de dados Suap

In [1]:
import pandas as pd
import plotly.express as px

#total geral = 10044; total livros = 4706; total geral sem livros = 5355

suap = pd.read_excel('suap.xlsx')
# print('suap info')
# display(suap.info())

priori= pd.read_excel('priori_completo.xlsx')

# capturando segunda planilha e removendo coluna observação
priori2 = pd.read_excel('priori_completo.xlsx', sheet_name=1)
priori2 = priori2.drop(columns='Observação')

# juntando planilhas
# priori utilizado em montar_compatarivo(suap_total, priori)
priori = pd.concat([priori, priori2])

priori_ED = priori['C. Custo'].drop_duplicates()
priori_ED = priori_ED.drop([6606, 6607])
# IFPE449052-99 IFPE449052-87 elementos de despesa dropados

priori_ED = priori_ED.sort_values(ascending=True)
#reindexando uutilizando list comprehension
priori_ED.index = [x for x in range(0, len(priori_ED))]

# display(priori.info())
# priori
# priori_livros = priori[priori['ED_priori'].str.contains('449052-18')]
# priori = priori[priori['ED_priori'].str.contains('449052-18')==False]
# display(priori.info())


In [2]:
livros1 = pd.read_table("parte 1 biblio.xls")
livros2 = pd.read_table("parte 2 biblio.xls")
livros3 = pd.read_table("parte 3 biblio.xls")
livros4 = pd.read_table("parte 4 biblio.xls")
livros5 = pd.read_table("parte 5 biblio.xls")

#limpando valores str para conversão da coluna em float
suap_livros = pd.concat([livros1, livros2, livros3, livros4, livros5])
suap_livros['VALOR'] = suap_livros['VALOR'].str.replace(".", "", regex=True)
suap_livros['VALOR'] = suap_livros['VALOR'].str.replace(",", ".", regex=True)

#recuprando base de dados completa para utilizar em gráficos e convertendo voluna valor em numero
suap_completo = pd.concat([suap, suap_livros])
suap_completo['VALOR'] = pd.to_numeric(suap_completo['VALOR'])

#suap_total utilizado em montar_compatarivo(suap_total, priori)
suap_total = pd.concat([suap, suap_livros])
suap_total['DATA DA ENTRADA'] = pd.to_datetime(suap_total['DATA DA ENTRADA'])

# dropando itens com entrada após abril 2021
suap_total = suap_total[suap_total['DATA DA ENTRADA'] < '2021-05-01']
# print(suap_total.info())

# dropando itens levantados virtualmento pela priori 
# suap_mod = suap_total[suap_total['DESCRICAO'].str.contains('MÓDULO DIDÁTICO,')]

suap_ED = suap_total['ED'].drop_duplicates()
suap_ED = suap_ED.sort_values(ascending=True)

#reindexando uutilizando list comprehension
suap_ED.index = [x for x in range(0, len(suap_ED))]
# print(len(suap_ED))

In [3]:
#função que varre o dataframe e remove cadeiras escolares

lista_drop = ['MÓDULO DIDÁTICO,', 'PLACA DE GESSO ACARTONADO', 'PORTA PARA DIVISÓRIA', 'Conjunto de guarda corpo medindo', 
             'PAR de corrimão em ferro', 'COBERTURA LEVE', 'KIT SALA MODULAR', 'CADEIRA ESCOLAR EM RESINA', 
             'CARTEIRA ESCOLAR - EM RESINA', 'CADEIRA PLASTICA UNIVERSAL',
             'CADEIRA ESCOLAR', 'CADEIRA COMUM. CADEIRA DE PLÁSTICO', 'MESA QUADRADA DE PLÁSTICO', 
             'POLTRONA PRANCHETA EM RESINA']

def dropar_descricao(data_frame, coluna, lista):

    for item in lista:
#         data_drop = data_frame[data_frame[coluna].str.contains(item)] 
        data_frame = data_frame[data_frame[coluna].str.contains(item)==False] #todas as linhas que não possuem o item
#         print(f'{item} - {len(data_drop)}')
    return data_frame

priori = dropar_descricao(priori, 'Descrição', lista_drop)
suap_total = dropar_descricao(suap_total, 'DESCRICAO', lista_drop)

# print(suap_total.info())

In [4]:
#CRUZAMENTO DOS DADOS ENTRE BASE DE DADOS SUAP E PRIORI COM A REMOÇÃO DE ITENS LEVANTADOS VIRTUALMENTE, CADEIRAS ESCOLARES POR EXEMPLO

suap_merge = suap_total[['NUMERO', 'SALA', 'DESCRICAO', 'CARGA ATUAL', 'ED', 'VALOR']]
priori_merge = priori[['Nr. Ativo', 'Nome Local', 'Descrição', 'C. Custo']]
priori_merge = priori_merge.rename(columns={'Nr. Ativo': 'NUMERO', 'Nome Local': 'SALA_priori', 'Descrição': 'DESCRICAO_priori', 'C. Custo': 'ED_priori'})
# display(priori_merge.info())
# display(suap_merge.info())

diferenca = pd.merge(suap_merge, priori_merge, on='NUMERO', how='outer', indicator=True)

diferenca_suap = diferenca[diferenca['_merge'].str.contains('left_only')]
diferenca_suap.index = [x for x in range(0, len(diferenca_suap))]
diferenca_suap = diferenca_suap.dropna(axis='columns')
diferenca_suap['VALOR'] = pd.to_numeric(diferenca_suap['VALOR'])

# display(diferenca_suap)
diferenca.to_excel('diferenca_total.xlsx', index=False)

In [15]:
# montar data frame pra o gráfico
def montar_comparativo(df_suap, df_priori):
    
    chart_df = pd.DataFrame()
    chart_df['ED'] = suap_ED
    suap_conta = []
    priori_conta = []
    porcentagem_suap = []

    for ed in chart_df['ED']:
        conta_s = df_suap[df_suap['ED'].str.contains(ed)]
        suap_conta.append(len(conta_s))
        conta_p = df_priori[df_priori['C. Custo'].str.contains(ed)]
        priori_conta.append(len(conta_p))


    chart_df['QTD_SUAP'] = suap_conta
    chart_df['QTD_PRIORI'] = priori_conta

#     chart_df['PC_SUAP'] = (chart_df['QTD_SUAP']/sum(chart_df['QTD_SUAP']))*100
#     chart_df['PC_PRIORI'] = (chart_df['QTD_PRIORI']/sum(chart_df['QTD_PRIORI']))*100

#     display(chart_df)

    total = {'ED': 'TOTAL', 'QTD_PRIORI': sum(priori_conta), 'QTD_SUAP': sum(suap_conta)}

    fig = px.bar(chart_df, x=chart_df['ED'], y=[chart_df['QTD_SUAP'], chart_df['QTD_PRIORI']], barmode='group')
    fig.update_layout(
        title="Cruzamento entre Levantamento Físico SUAP x PRIORI",
        xaxis_title="Elemento de Despesa",
        yaxis_title="Quantidades (unidades)",
        legend_title="Legenda"
        )
    fig.write_html('charts/Cruzamento entre Levantamento Físico SUAP x PRIORI.html')
    fig.show()
    
    piz = px.pie(chart_df, values=chart_df['QTD_SUAP'], names=chart_df['ED'])
    piz.update_layout(title="Proporção Quantidade/ED no SUAP", legend_title="Elemento de Despesa (ED)")
    piz.show()
    
    piz2 = px.pie(chart_df, values=chart_df['QTD_PRIORI'], names=chart_df['ED'])
    piz2.update_layout(title="Proporção Quantidade/ED PRIORI", legend_title="Elemento de Despesa (ED)")
    piz2.show()
    # chart_df = chart_df.append(total, ignore_index=True)

In [11]:
def montar_totalizacao(df_suap_completo):

    chart_completo = pd.DataFrame()
    chart_completo['ED'] = suap_ED
    quantidade_ed = []
    somatorio_ed = []

    
    for ed in chart_completo['ED']:
        qtd = df_suap_completo[df_suap_completo['ED'].str.contains(ed)]
        quantidade_ed.append(len(qtd))
        somatorio_ed.append(sum(qtd['VALOR']))
        

    # for ed in suap_completo['ED']:


    chart_completo['QTD_TOTAL'] = quantidade_ed
    chart_completo['SOMATORIO'] = somatorio_ed

    cont = px.bar(chart_completo, x=chart_completo['ED'], y=chart_completo['QTD_TOTAL'])
    cont.update_layout(
        title="Totalização por Elemento de Despesa no SUAP até 01/09/2021",
        xaxis_title="Elemento de Despesa",
        yaxis_title="Quantidades (unidades)",
        legend_title="Legenda"
        )
    cont.show()
    
    qtdpizz = px.pie(chart_completo, values=chart_completo['QTD_TOTAL'], names=chart_completo['ED'])
    qtdpizz.update_layout(title="Proporção QTD/ED no SUAP", legend_title="Elemento de Despesa (ED)")
    qtdpizz.show()
    
    fin = px.bar(chart_completo, x=chart_completo['ED'], y=chart_completo['SOMATORIO'])
    fin.update_layout(
        title="Valores por Elemento de Despesa no SUAP até 01/09/2021",
        xaxis_title="Elemento de Despesa",
        yaxis_title="Valor (Milhões de Reais)",
        legend_title="Legenda"
        )
    fin.show()
    
    finpizz = px.pie(chart_completo, values=chart_completo['SOMATORIO'], names=chart_completo['ED'])
    finpizz.update_layout(title="Proporção Valor/ED no SUAP", legend_title="Elemento de Despesa (ED)")
    finpizz.show()

In [19]:
# antes de montar comparativo rodar a função droprar_descrição
montar_comparativo(suap_total, priori)
montar_totalizacao(suap_completo)
# montar_totalizacao(diferenca_suap)